### GENETIC ALGORITHM IMPLEMENTATION FOR SOLVING 8 QUEENS.

In [19]:
# Import the Basic Libraries
import pandas as pd
import numpy as np

import random as random

In [20]:
# Soln Class.. Fitness of each Queen's position will be calculated and Scored, when a new Instance is created.
class Soln:
    def __init__(self, Solution):
        self.Solution = Solution
        self.FitFlags = self.FitnessCalc() 
        self.Score  = np.sum(self.FitFlags)
        
    def FitnessCalc(self):
            Row = [0, 1, 2, 3, 4, 5, 6, 7]
            FitFlags = [0, 0, 0, 0, 0, 0, 0, 0]
            
            for R in range(N):
                PositionFitScore = 0
                for C in range(N):
                    if ((Row[R] != Row[C]) & (self.Solution[R] != self.Solution[C]) & (abs(Row[R] - Row[C]) != abs(self.Solution[R] - self.Solution[C]))):
                        PositionFitScore += 1
                if PositionFitScore == 7:
                    FitFlags[R] = 1
            
            return FitFlags

In [21]:
# Function to populate Initial Solution Sets
def SolnSample():
    Sample_Soln = []
    i = 0
    while i < N:
        r=random.randint(0,7)
        if r not in Sample_Soln:
            Sample_Soln.append(r)
            i += 1
    return Sample_Soln

In [22]:
# Populate Sample Solutions
def Populate():
    Population = []
    for i in range(PopulationCount):
        Sample = SolnSample()
        Population.append(Soln(Sample))
    
    # Sort the Solution Population on Descending order of Fitness Score
    Population.sort(key=lambda x: x.Score, reverse=True)
    return Population

In [23]:
# Removal of Duplicates introduced by Crossover, if any
def RemoveDups(Child_Arr):
    
    # Find the Unique values and no. of occurances of each Unique value
    Uniques,Counts = np.unique(Child_Arr,return_counts=1)
    
    # If Unique values Equal N, there are No Duplicates
    if (len(Uniques) != N):
        # Mask the positions of Duplicates
        Masked = np.isin(Child_Arr,Uniques[Counts>1])
        # Identify the Missing values in Child
        Missing_Values = np.setdiff1d(np.arange(len(Child_Arr)),Child_Arr[~Masked])
        # Shuffle the Missing values and insert them in Masked positions
        np.random.shuffle(Missing_Values)
        j = 0
        for i in range(N):
            if (Masked[i] == True):
                Child_Arr[i] = Missing_Values[j]
                j +=1
    
    return Child_Arr

In [24]:
# Crossover Function - Simple 1-Point Crossover, with Removal of Duplicates introduced, if any 
def Crossover(Parent1, Parent2):
    
    Child_Arr=np.array([0, 0, 0, 0, 0, 0, 0, 0])
    
    # Randomly Identify the Corssover point
    CrossOverPoint = random.randint(2,5)
    
    # 1-Point Crossover
    for i in range(N):
        if i <= CrossOverPoint: 
            Child_Arr[i] = Parent1[i]
        else:
            Child_Arr[i] = Parent2[i]
            
    # Clean up the Duplicates, if any, introduced by Crossover
    Child_Arr = RemoveDups(Child_Arr)
    Child = Child_Arr.tolist()
    
    return Child

In [25]:
# Mutation Function - Swap the positions of Queen
def Mutate(Soln, Loop):
    
    # With Increase in Iterations, Increase the N. of Bits to be Mutated
    Reps = int((Loop+100)/100)
    
    # Limiting the Mutation to max 6 Swaps
    if Reps > 6:
        Reps = 6
    
    for i in range(Reps):
        # Randomly identify the Position to be Mutated
        MutationPoint = random.randint(1,7)
        
        # And Swap the positions
        Temp = Soln[MutationPoint]
        Soln[MutationPoint] = Soln[MutationPoint-1]
        Soln[MutationPoint-1] = Temp
    
    return Soln

In [26]:
# Selection, Crossover and Mutation Loop
def Iterate(PopulationData):
    Final_Solution = []
    Iterations = 0
    for loop in range(MaxGenerations):
    
        if (PopulationData[0].Score == 8):
            Final_Solution = PopulationData[0].Solution
            Iterations = loop
            break
        if (PopulationData[1].Score == 8):
            Final_Solution = PopulationData[0].Solution
            Iterations = loop
            break
    
        #print('********************************** Generation ',loop, " *****************************************")
    
        # Select TWO Fittest Solutions from the Population
        Selection1 = PopulationData[0].Solution
        Selection2 = PopulationData[1].Solution
        #print("S ->", PopulationData[0].Solution, " ", PopulationData[0].FitFlags, " || ", PopulationData[1].Solution, " ", PopulationData[1].FitFlags)
    
        # Call Cross Over Function for creating Children
        Child1 = Soln(Crossover(Selection1, Selection2))
        Child2 = Soln(Crossover(Selection2, Selection1))
        #print("C ->", Child1.Solution, " ", Child1.FitFlags, " || ", Child2.Solution, " ", Child2.FitFlags)

        # Call Mutation Function For Children
        MutChild1 = Soln(Mutate(Child1.Solution, loop))
        MutChild2 = Soln(Mutate(Child2.Solution, loop))
        #print("M ->", MutChild1.Solution, " ", MutChild1.FitFlags, " || ", MutChild2.Solution, " ", MutChild2.FitFlags)
        
        # Append the Mutated Children to existing Population
        PopulationData.append(MutChild1)
        PopulationData.append(MutChild2)
        PopulationData.sort(key=lambda x: x.Score, reverse=True)
        PopulationCount = len(PopulationData)

    return Final_Solution, loop

In [27]:
# Main Section - Set the Count for Initial Population

N = 8
PopulationCount = 50
MaxGenerations = 1000

# Limited the Max Generations to 1000. 
# On rare ocasions, if Generation count reaches this limit in Trail 1, reset the population and start a fresh Trail:

Trial = 1
while True:
    PopulationData = Populate()
    Output, Gen_Count = Iterate(PopulationData)
    if Gen_Count != 999:
        print("Trial # : ", Trial, " No. of Generations : ", Gen_Count, " Solution -> ", Output)
        break
    Trial += 1

Trial # :  1  No. of Generations :  19  Solution ->  [4, 0, 7, 3, 1, 6, 2, 5]
